In [1]:
import countergen

In [2]:
dsa = countergen.Dataset.from_default("doublebind-likable")
dsb = countergen.Dataset.from_default("doublebind-unlikable")
ds2a = countergen.Dataset.from_default("female-stereotypes")
ds2b = countergen.Dataset.from_default("male-stereotypes")

In [3]:
augmenters = [countergen.SimpleAugmenter.from_default("gender")]

In [4]:
countergen.config.seed(0)
aug_dsa = dsa.augment(augmenters)
aug_dsb = dsb.augment(augmenters)
aug_ds2a = ds2a.augment(augmenters)
aug_ds2b = ds2b.augment(augmenters)

100%|██████████| 30/30 [00:00<00:00, 58525.17it/s]


In [5]:
from countergen import get_generative_model_evaluator, api_to_generative_model

model_names = ["ada", "babbage", "curie", "davinci", "instruct"]
model_codes = ["ada","babbage","curie", "davinci", "text-davinci-003"]
model_evs = [get_generative_model_evaluator(api_to_generative_model(code), "probability") for code in model_codes]

In [6]:
model_codes

['ada', 'babbage', 'curie', 'davinci', 'text-davinci-003']

In [7]:
aggregator = countergen.aggregators.DifferenceStats("male", "female", relative_difference=True)

In [8]:
resultsa = [countergen.evaluate(aug_dsa.samples, model_ev, aggregator) for model_ev in model_evs]
resultsb = [countergen.evaluate(aug_dsb.samples, model_ev, aggregator) for model_ev in model_evs]
results2a = [countergen.evaluate(aug_ds2a.samples, model_ev, aggregator) for model_ev in model_evs]
results2b = [countergen.evaluate(aug_ds2b.samples, model_ev, aggregator) for model_ev in model_evs]

100%|██████████| 30/30 [00:48<00:00,  1.63s/it]


In [11]:
print("female stereotype", "male stereotype", "doublebind likable", "doublebind unlikable")

def process(result):
    significant = "*" if abs(result.mean) > result.uncertainty_2sig else ""
    return f"{result.mean:.2f}{significant}"

for i, name in enumerate(model_names):
    print(name, ":", process(results2a[i]), process(results2b[i]), process(resultsa[i]), process(resultsb[i]))

female stereotype male stereotype doublebind likable doublebind unlikable
ada : -0.23 0.18 0.07 -0.26*
babbage : -0.13 0.26* -0.06 -0.06
curie : -0.21 0.31* 0.12 -0.11
davinci : -0.24* 0.30* -0.19 -0.35*
instruct : 0.16 0.51* -0.29* -0.46*
